In [1]:
!pip install mlflow scikit-learn pandas matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.0/684.0 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.7 MB/s eta 0:00:00


In [4]:
import zipfile

with zipfile.ZipFile("/content/archive.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/f1_data")


In [5]:
import os

files = os.listdir("/content/f1_data")
files


['lap_times.csv',
 'drivers.csv',
 'sprint_results.csv',
 'constructors.csv',
 'status.csv',
 'pit_stops.csv',
 'constructor_standings.csv',
 'circuits.csv',
 'constructor_results.csv',
 'races.csv',
 'driver_standings.csv',
 'results.csv',
 'qualifying.csv',
 'seasons.csv']

In [6]:
import pandas as pd

lap_times = pd.read_csv("/content/f1_data/lap_times.csv")
lap_times.head()


,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342


In [7]:
from sklearn.model_selection import train_test_split

# Use driverId, lap, and position as features
X = lap_times[["driverId", "lap", "position"]]
y = lap_times["milliseconds"]

# Split into train/test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

def run_experiment(n_estimators, max_depth, random_state):
    with mlflow.start_run():
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=random_state
        )
        model.fit(X_train, y_train)
        preds = model.predict(X_test)

        # calculate metrics
        mse = mean_squared_error(y_test, preds)
        mae = mean_absolute_error(y_test, preds)
        r2 = r2_score(y_test, preds)

        # log parameters and metrics
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("random_state", random_state)

        mlflow.log_metric("mse", mse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)

        # save model
        mlflow.sklearn.log_model(model, "model")

        # plot predictions vs actual values
        plt.figure(figsize=(6, 4))
        plt.scatter(y_test, preds, alpha=0.4)
        plt.xlabel("True lap time (ms)")
        plt.ylabel("Predicted lap time (ms)")
        plt.title("Predicted vs True")
        plt.tight_layout()
        plt.savefig("prediction_plot.png")
        mlflow.log_artifact("prediction_plot.png")
        plt.close()


In [9]:
for n in [50, 100, 150]:
    for depth in [5, 10, 15]:
        run_experiment(n_estimators=n, max_depth=depth, random_state=1)


2025/04/09 01:29:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/04/09 01:30:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/04/09 01:31:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/04/09 01:32:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/04/09 01:33:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/04/09 01:34:39 

In [10]:
# Install pyngrok
!pip install pyngrok --quiet

# Set up a temporary tracking URI folder
import os
os.environ["MLFLOW_TRACKING_URI"] = "file:/content/mlruns"

# Start MLflow UI in background
get_ipython().system_raw("mlflow ui --port 5000 &")


In [13]:
!ngrok config add-authtoken 2vTJ1u9zUtkUyt9zChYGoTHKogr_7ipj9Y3yXcNG53y4wGEes


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [14]:
from pyngrok import ngrok

public_url = ngrok.connect(5000)
print("MLflow UI is available at:", public_url)


MLflow UI is available at: NgrokTunnel: "https://2c50-34-71-121-120.ngrok-free.app" -> "http://localhost:5000"
